In [1]:
import os
import numpy as np
import pandas as pd
from time import time
from utils import aspect_tools

data_dir = "/home/stavros/DATA/AirbnbReviews"
#data_dir = "D:/"

In [3]:
area_dir = os.path.join(data_dir, "athens")
reviews = pd.read_pickle(os.path.join(area_dir, "reviews_with_aspects_379118samples.pkl"))
print(reviews.shape)
reviews = reviews[pd.notnull(reviews["listing_id"])]
reviews = reviews[pd.notnull(reviews["comments"])]
print(reviews.shape)

(379118, 8)
(294403, 8)


In [4]:
reviews["listing_id"].value_counts()

1177492     554
1079291     520
2306865     507
1484797     464
3431705     464
           ... 
34097757      1
13617762      1
24426143      1
32656017      1
19618064      1
Name: listing_id, Length: 8731, dtype: int64

In [5]:
top_listing_reviews = reviews[reviews["listing_id"] == 1177492]
print(top_listing_reviews.shape)

(554, 8)


In [11]:
(top_listing_reviews["aspects"].map(lambda x: len(x) > 0)).sum()

538

In [25]:
import collections
all_listing_aspects = collections.Counter()
for aspects in top_listing_reviews.aspects:
    for phrase, score in aspects.items():
        to_add = 2 * int(score > 0) - 1
        for word in phrase.split(" "):
            all_listing_aspects[word] += to_add

In [26]:
negative_listing_aspects = collections.Counter({k: -v for k, v in all_listing_aspects.items() if v < 0})

In [27]:
negative_listing_aspects.most_common()

[('issues', 5),
 ('break', 5),
 ('stairs', 3),
 ('problem', 3),
 ('trouble', 3),
 ('ferry', 2),
 ('strike', 2),
 ('steps', 2),
 ('mobility', 2),
 ('ruins', 2),
 ('problems', 2),
 ('bug', 2),
 ('event', 1),
 ('city', 1),
 ('traffic', 1),
 ('noise', 1),
 ('difficulties', 1),
 ('mishap', 1),
 ('ordeal', 1),
 ('photos', 1),
 ('lightning', 1),
 ('transportation', 1),
 ('heat', 1),
 ('steal', 1),
 ('word', 1),
 ('difficulty', 1),
 ('drawbacks', 1),
 ('lemon', 1),
 ('nothing', 1),
 ('plaka', 1),
 ('drawback', 1),
 ('points', 1),
 ('shame', 1),
 ('case', 1),
 ('accessibility', 1),
 ('concerns', 1),
 ('worry', 1),
 ('cons', 1),
 ('struggles', 1),
 ('niggles', 1),
 ('tiring', 1),
 ('close', 1),
 ('mintue', 1),
 ('taxis', 1),
 ('flight', 1),
 ('noisy', 1),
 ('shamelessness', 1),
 ('consequences', 1),
 ('staircase', 1),
 ('signage', 1),
 ('travel', 1),
 ('meals', 1),
 ('staff', 1),
 ('menu', 1),
 ('none', 1),
 ('baby', 1),
 ('cockroach', 1),
 ('bottle', 1),
 ('traveller', 1),
 ('dog', 1),
 ('enjoy

In [17]:
def has_negative_aspect(aspects):
    for v in aspects.values():
        if v < 0:
            return True
    return False

negative_comments = top_listing_reviews[top_listing_reviews["aspects"].map(has_negative_aspect)]
print(negative_comments.shape)

(92, 8)


In [36]:
def make_word_aspects(aspects):
    new_aspects = collections.Counter()
    for aspect, score in aspects.items():
        for word in aspect.split(" "):
            new_aspects[word] = score
    return new_aspects

In [37]:
top_listing_reviews["word_aspects"] = top_listing_reviews["aspects"].map(make_word_aspects)

/home/stavros/PythonEvironments/nlp-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
top_listing_reviews[top_listing_reviews["word_aspects"].map(lambda x: "cockroach" in x)]

,listing_id,id,date,reviewer_id,reviewer_name,comments,processed_comments,aspects,word_aspects
245412,1177492,174520205,2017-07-26,32385519,Heather,We unfortunately only spent one of three night...,We unfortunately only spent one of three night...,"{'bug': -1, 'baby cockroach': -1, 'location': ...","{'bug': -1, 'baby': -1, 'cockroach': -1, 'loca..."


In [50]:
top_listing_reviews.loc[245412]["comments"]

'We unfortunately only spent one of three nights that we had planned at this apartment. We found two live cockroaches in the bathroom, and then a dead bug in the bed which strongly resembled photos of bedbugs. (We also both had bites after our stay). While bedbugs can happen anywhere, it was very concerning to me that the host did not take the situation seriously. He claimed the bug was a baby cockroach, and offered no type of solution for either type of bug.\n\nThe apartment was in a great location, on a quiet street, and Rio was quick to answer our messages. If he deals with the bug issues, this would be a nice apartment to stay in. Unfortunately, that is the best review I can give.\n\n(It should be noted that Rio gave a full refund for the two nights which we did not stay (but not for the first night). However, we lost a half day of our vacation in Athens, spent at the laundromat washing our things.)'